<a href="https://colab.research.google.com/github/syafiyahh/MidtermTask/blob/main/MidtermTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Syafiyah Hamnah Hasan
NPM: 2106726144

In [1]:
# Clone YOLOv5 repository and install dependencies
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.62 MiB | 21.18 MiB/s, done.
Resolving deltas: 100% (11695/11695), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 16.4 MB/s eta 0:00:00


In [2]:
import torch          # For loading and running YOLOv5 model
import cv2            # For video processing
import json           # For saving and loading detection results


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Set the path to the beach.mp4 video file in Google Drive
video_path = '/content/drive/MyDrive/beach.mp4'


Mounted at /content/drive


In [4]:
# Load YOLOv5 model (small version for efficiency)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)


/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-11-2 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 253MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [5]:
# Open the video file
video_capture = cv2.VideoCapture(video_path)

# List to store detection results
detections = []

while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    # Perform object detection on the frame
    results = model(frame)

    # Extract detection information for each object
    for detection in results.pred[0]:  # results.pred[0] contains detections for the frame
        x1, y1, x2, y2, confidence, class_id = detection
        label = model.names[int(class_id)]

        # Append detection details to the list
        detections.append({
            "frame": int(video_capture.get(cv2.CAP_PROP_POS_FRAMES)),
            "label": label,
            "confidence": float(confidence),
            "bbox": [int(x1), int(y1), int(x2), int(y2)]
        })

video_capture.release()

# Save detections to JSON file for future analysis
with open('detections_beach.json', 'w') as f:
    json.dump(detections, f)

print("Object detection completed. Results saved to 'detections_beach.json'")


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
WARNING ⚠️ NMS time limit 0.550s exceeded
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/co

Object detection completed. Results saved to 'detections_beach.json'


In [6]:
# Re-open the original video for processing
video_capture = cv2.VideoCapture(video_path)
output_path = 'output_with_detections.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(video_capture.get(cv2.CAP_PROP_FPS))
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Process each frame and add detections
frame_number = 0
while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    # Filter detections for the current frame
    frame_detections = [d for d in detections if d["frame"] == frame_number]

    # Draw bounding boxes and labels on the frame
    for det in frame_detections:
        x1, y1, x2, y2 = det["bbox"]
        label = f"{det['label']} ({det['confidence']:.2f})"

        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Add label
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Write the frame to the output video
    out.write(frame)
    frame_number += 1

video_capture.release()
out.release()

print("Video with detections saved as 'output_with_detections.mp4'")


Video with detections saved as 'output_with_detections.mp4'


In [7]:
from google.colab import files
files.download('output_with_detections.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
!cp output_with_detections.mp4 /content/drive/MyDrive/output_with_detections.mp4


In [9]:
import cv2
import torch
from collections import defaultdict

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Define the path to the video file
video_path = 'output_with_detections.mp4'  # Replace with actual path if necessary

# Open the video file
video_capture = cv2.VideoCapture(video_path)

# Dictionary to store the count of each class
object_counts = defaultdict(int)

# Process each frame and count detected objects
while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    # Perform object detection on the frame
    results = model(frame)

    # Extract detection information for each object
    for detection in results.pred[0]:  # results.pred[0] contains detections for the frame
        x1, y1, x2, y2, confidence, class_id = detection[:6]
        label = model.names[int(class_id)]

        # Increment the count for each detected object class
        object_counts[label] += 1

video_capture.release()

# Display total counts for each class detected in the video
print("Total counts for each object class:")
for label, count in object_counts.items():
    print(f"{label}: {count}")

# Get the count of people specifically
person_count = object_counts.get("person", 0)
print(f"\nTotal number of persons detected: {person_count}")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-2 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/commo

Total counts for each object class:
person: 5044
dog: 118
umbrella: 517
surfboard: 411
horse: 11
kite: 34
elephant: 2
chair: 88
boat: 2
backpack: 5
bird: 1

Total number of persons detected: 5044


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [10]:
import matplotlib.pyplot as plt

# Initialize list to store person count per frame
person_counts_per_frame = []

# Re-open the video file to process from the beginning
video_capture = cv2.VideoCapture(video_path)

# Process each frame and track the number of people detected
frame_number = 0
while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    # Perform object detection on the frame
    results = model(frame)

    # Count the number of people detected in this frame
    person_count = sum(1 for detection in results.pred[0] if model.names[int(detection[5])] == "person")
    person_counts_per_frame.append(person_count)

    frame_number += 1

video_capture.release()

# Plot person counts per frame
plt.plot(person_counts_per_frame, label="Persons Detected per Frame")
plt.xlabel("Frame Number")
plt.ylabel("Person Count")
plt.title("Person Detection Counts per Frame")
plt.legend()
plt.show()


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.am